## Import Libraries

In [1]:
# data processing
import pandas as pd

#linear algebra
import numpy as np

# data visualization
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

#regex
import re

#Algorithms

## Getting the Data

In [2]:
#online_coupon train data
df_on=pd.read_csv("ccf_online_stage1_train.csv")

#offline_coupon train data
df_off=pd.read_csv("ccf_offline_stage1_train.csv")

#offline_coupon test data
df_off_test=pd.read_csv("ccf_offline_stage1_test_revised.csv")

## Feature Engineering

In [3]:
#creating training dataset to fed to the model

x = 'Null'
df_off_unique = (df_off.fillna(x).groupby(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date']).size().reset_index().rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_off_unique.drop(['Count'],axis=1,inplace=True)


x = 'Null'    
df_on_unique = (df_on.fillna(x).groupby(['User_id', 'Merchant_id', 'Action', 'Coupon_id', 'Discount_rate',
                'Date_received', 'Date']).size().reset_index().rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_on_unique.drop(['Count'],axis=1,inplace=True)

## User level features

In [ ]:
df_off_unique2=df_off_unique[df_off_unique['Date_received'].isna()==False].copy(deep=True)
df_off_unique2['Date'] = pd.to_datetime(df_off_unique2['Date'],format="%Y%m%d")
df_off_unique2['Date_received'] = pd.to_datetime(df_off_unique2['Date_received'],format="%Y%m%d")

df_off_unique2["Redeem_date"]=df_off_unique2["Date"].dt.date-df_off_unique2["Date_received"].dt.date
df_off_unique2["Redeem_date"]=df_off_unique2["Date"].dt.date-df_off_unique2["Date_received"].dt.date
df_off_unique2['Redeem_date'] = df_off_unique2['Redeem_date'].dt.days.astype('str')
df_off_unique2['Redeem_date'] = pd.to_numeric(df_off_unique2['Redeem_date'],errors="coerce")
split_data = df_off_unique2["Discount_rate"].str.split(":")
data = split_data.to_list()
for i in range(len(data)):
    if len(data[i])==1:
        data[i].insert(1,100-float(data[i][0])*100)
        data[i][0]=100
        

df_off_unique2['temp']=data
df_off_unique2[['Original_Price','Discounted_price']] = pd.DataFrame(df_off_unique2.temp.values.tolist(), index= df_off_unique2.index)
df_off_unique2["Original_Price"]=df_off_unique2["Original_Price"].astype(float)
df_off_unique2["Discounted_price"]=df_off_unique2["Discounted_price"].astype(float)
df_off_unique2["Rate"]=round((df_off_unique2["Original_Price"]-df_off_unique2["Discounted_price"])/df_off_unique2["Original_Price"],3)
del df_off_unique2["temp"] 
df_off_unique2["Redeem_date"].fillna(-1, inplace = True) 
df_off_unique2["Distance"].fillna(df_off_unique2["Distance"].mean(), inplace = True) 
df_off_unique2["Target"]=0
df_off_unique2.loc[(df_off_unique2["Redeem_date"]<=15) & (df_off_unique2["Redeem_date"]>=0), 'Target'] = 1 
df_off_unique2

In [ ]:
# no of coupons users received, redeemed and redemption rate
def create_user_crr(df):
    df2=df[df["Coupon_id"].isna()==False]
    df3=df2.groupby('User_id').agg({"Coupon_id":"count","Date":"count"})
    df3.rename(columns = {"Coupon_id": "Coupon_receive_count","Date":"Coupon_redeem_count"}, inplace = True) 
    df3["User_Redeem_Receive_Ratio"]=df3["Coupon_redeem_count"]/df3["Coupon_receive_count"]
    df3.reset_index(inplace=True)
    df3.sort_values(by=["Coupon_receive_count"],ascending=False,inplace=True)
    df=pd.merge(df,df3)
    return df

#avg distance of user from store
def create_avg_dist_user(df):
    df2=df_off_unique2.groupby('User_id').agg({"Distance":"mean"})
    df2.rename(columns = {"Distance": "User_avg_distance"}, inplace = True) 
    df2.reset_index(inplace=True)
    df=pd.merge(df,df2)
    return df

# no of different actions by users
def create_no_of_actions(df_off_unique2,df_on_unique,df_off_unique):
    list_on_off = set(df_on_unique["User_id"].unique()).intersection(set(df_off_unique["User_id"].unique()))
    df_on_off = df_on[df_on['User_id'].isin(list(list_on_off))]
    df=pd.get_dummies(df_on_off["Action"])
    df_on_off=pd.concat([df_on_off,df],axis=1)
    df_on_off.rename(columns = {0: "No_of_Clicks",1:"No_of_Buy",2:"No_of_Get_Coupons"}, inplace = True) 
    df2=df_on_off.groupby('User_id').agg({"No_of_Clicks":"count","No_of_Buy":"count","No_of_Get_Coupons":"count"})
    #df2["Buy_Click_Ratio"]=df2["No_of_Buy"]/df2["No_of_Clicks"]
    df2.reset_index(inplace=True)
    df_off_unique2=pd.merge(df_off_unique2,df2)
    return df_off_unique2

In [ ]:
df_off_unique2=create_user_crr(df_off_unique2)
df_off_unique2=create_avg_dist_user(df_off_unique2)
#df_off_unique2=create_no_of_actions(df_off_unique2,df_on_unique,df_off_unique)
df_off_unique2

In [ ]:
df_off_unique2

In [ ]:
df_off_unique2.to_csv("Train_data.csv")